# Counting Stability Chambers of Moduli Spaces

This notebook shows how this partitioning algorithm is useful in **multiple areas of interest**.
In particular, it has been used in **state-of-the-art research on moduli spaces of parabolic vector bundles**,
where it allowed us to **count the number of stability chambers** of these moduli spaces with parameters $n$ (number of stability points) and $r$ (rank of vector bundles).

For background see:

> [_A computational analysis of isomorphism classes of moduli spaces of parabolic vector bundles_](https://repositorio.comillas.edu/xmlui/handle/11531/89959)


## Step 1. Imports and Setup


In [1]:
from time import perf_counter

import sys
from pathlib import Path

# Add the project root (parent of "nb") to sys.path
project_root = Path("..").resolve()
sys.path.insert(0, str(project_root))

from polypart.geometry import Polytope, Hyperplane
from polypart.ppart import build_partition_tree
from polypart.ftyping import Fraction, as_fraction_vector
from polypart.io import save_tree


## Step 2. Build the Polytope

In a moduli space of parabolic vector bundles, the ambient space is a product of simplices of dimension $n(r-1)$.
We can build this polytope using the predefined function `polypart.polytopes.get_simplex`.

We will study the case of $n=1$ and $r=7$, which gives a simplex of dimension $6$ with $7$ facets.


In [2]:
from polypart.polytopes import get_simplex

n, r = 1, 7
simplex = get_simplex(r - 1)
simplex.extreme()
simplex

Polytope(dim=6, n_ineq=7, n_vertices=7)

## Step 3. Generate Hyperplane Arrangement

In the moduli space classification problem, there are some hyperplanes called "stability walls" that partition the parameter space (starting polytope) into chambers, where each chamber represents a different moduli space.

We will generate these hyperplanes using the predefined function `polypart.arrangements.get_moduli_arrangement`, which implements the combinatorial construction of these hyperplanes for the given parameters $n$, $r$, and $d$ (degree of the vector bundles, which we set to $0$ for simplicity).


In [3]:
from polypart.arrangements import get_moduli_arrangement

hyperplanes = get_moduli_arrangement(n, r, d=0)
print(f"Generated arrangement with {len(hyperplanes)} hyperplanes.")

Generated arrangement with 65 hyperplanes.


## Step 4. Build the Partition Tree

We now apply the partitioning algorithm to the polytope and the set of hyperplanes using to different strategies: 'random' and 'v-entropy'. We compare the runtime and the maximum and average depth of the resulting trees.


- 'Random' strategy: selects hyperplanes randomly at each node.


In [4]:
start = perf_counter()
tree, n_chambers = build_partition_tree(simplex, hyperplanes, strategy="random")
elapsed = perf_counter() - start
print(f"Found {n_chambers} chambers in {elapsed:.2f} s")

import json

print("Partition tree statistics:")
print(json.dumps(tree.stats(), indent=4))

Found 1000 chambers...
Found 1296 chambers in 3.16 s
Partition tree statistics:
{
    "total_nodes": 2591,
    "max_depth": 24,
    "avg_depth": 15.059413580246913,
    "per_depth_nodes": {
        "0": 1,
        "1": 2,
        "2": 4,
        "3": 6,
        "4": 6,
        "5": 12,
        "6": 24,
        "7": 42,
        "8": 74,
        "9": 116,
        "10": 164,
        "11": 232,
        "12": 276,
        "13": 260,
        "14": 236,
        "15": 240,
        "16": 240,
        "17": 170,
        "18": 122,
        "19": 94,
        "20": 76,
        "21": 84,
        "22": 68,
        "23": 30,
        "24": 12
    },
    "avg_candidates": 2.5584716325742956,
    "per_depth_avg_candidates": {
        "0": 65.0,
        "1": 64.0,
        "2": 46.0,
        "3": 29.666666666666668,
        "4": 35.333333333333336,
        "5": 23.833333333333332,
        "6": 15.0,
        "7": 10.523809523809524,
        "8": 6.837837837837838,
        "9": 4.862068965517241,
        "10

- 'V-entropy' strategy: maximizes volume entropy (approximated with number of vertices in each sub-polytope).


In [5]:
start = perf_counter()
tree, n_chambers = build_partition_tree(
    simplex, hyperplanes, strategy="v-entropy", remove_redundancies=True
)
elapsed = perf_counter() - start
print(f"Found {n_chambers} chambers in {elapsed:.2f} s")

# import json and print stats as pretty json
import json

print(json.dumps(tree.stats(), indent=4))

Found 1000 chambers...
Found 1296 chambers in 1.83 s
{
    "total_nodes": 2591,
    "max_depth": 14,
    "avg_depth": 11.247685185185185,
    "per_depth_nodes": {
        "0": 1,
        "1": 2,
        "2": 4,
        "3": 8,
        "4": 16,
        "5": 32,
        "6": 64,
        "7": 124,
        "8": 232,
        "9": 370,
        "10": 480,
        "11": 480,
        "12": 420,
        "13": 264,
        "14": 94
    },
    "avg_candidates": 2.074488614434581,
    "per_depth_avg_candidates": {
        "0": 65.0,
        "1": 64.0,
        "2": 46.0,
        "3": 31.0,
        "4": 22.375,
        "5": 14.5,
        "6": 8.96875,
        "7": 5.67741935483871,
        "8": 3.2672413793103448,
        "9": 1.972972972972973,
        "10": 1.1958333333333333,
        "11": 0.7791666666666667,
        "12": 0.3952380952380952,
        "13": 0.18181818181818182,
        "14": 0.0
    },
    "avg_inequalities_per_node": 7.947510613662678,
    "per_depth_avg_inequalities": {
        "

In [6]:
import cProfile

cProfile.run(
    """
tree, n_chambers = build_partition_tree(
    simplex, hyperplanes, strategy="v-entropy", remove_redundancies=True
)
""",
    "moduli_stability_chambers_profile_n1_r7_3.prof",
)

Found 1000 chambers...


## Step 6. Saving the Tree (Optional)

The decision tree is very useful in research for identifying **isomorphisms** between stability chambers, since it allows for efficient point location queries, and it also allows for a detailed analysis of the structure of the partition.


In [7]:
# save_tree(tree, f"moduli_n{n}_r{r}_v_entropy.json")
